In [56]:
!pip install pymango

In [57]:
from pymongo import MongoClient

In [58]:
# Global variable for the MongoDB connection string
connection_string = "mongodb://mongo:27017/"

def insert_data_to_mongodb(data_list, db_name, collection_name):
    """
    Insert a given collections to mongoDB.
    """
    with MongoClient(connection_string) as client:
        db = client[db_name]
        collection = db[collection_name]
        collection.insert_many(data_list)

In [59]:
# Insert persons/actors
persons = [
    {'id': 1, 'name': 'Charlie Sheen'}, 
    {'id': 2, 'name': 'Michael Douglas'}, 
    {'id': 3, 'name': 'Martin Sheen'}, 
    {'id': 4, 'name': 'Morgan Freeman'}
]
insert_data_to_mongodb(persons, "moviedb", "persons")

# Insert roles
roles = [
  { "person_id": 1, "movie_id": 1, "role":["Bud Fox"]},
  { "person_id": 2, "movie_id": 1, "role":["Carl Fox"]},
  { "person_id": 3, "movie_id": 1, "role":["Gordon Gekko"]},
  { "person_id": 2, "movie_id": 2, "role":["A.J. MacInerney"]},
  { "person_id": 3, "movie_id": 2, "role":["President Andrew Shepherd"]},
  { "person_id": 4, "movie_id": 3, "role":["Ellis Boyd 'Red' Redding"]}
]
insert_data_to_mongodb(roles, "moviedb", "roles")

# Insert movies
movies = [
  { "id": 1, "title": "Wall Street", "country":"USA","year":1987},
  { "id": 2, "title": "The American President", "country":"USA","year":1995},
  { "id": 3, "title": "The Shawshank Redemption", "country":"USA","year":1994},
]
insert_data_to_mongodb(movies, "moviedb", "movies")

In [60]:
client = MongoClient(connection_string)
db = client['moviedb']
collection = db['persons']
    
# Get all actors
actors = collection.find({})
for actor in actors:
    print(actor)

{'_id': ObjectId('652d41847230280569ccd2ab'), 'id': 1, 'name': 'Charlie Sheen'}
{'_id': ObjectId('652d41847230280569ccd2ac'), 'id': 2, 'name': 'Michael Douglas'}
{'_id': ObjectId('652d41847230280569ccd2ad'), 'id': 3, 'name': 'Martin Sheen'}
{'_id': ObjectId('652d41847230280569ccd2ae'), 'id': 4, 'name': 'Morgan Freeman'}


In [61]:
# Create a query to find actors with names starting with 'C'
query = {"name": {"$regex": "^C", "$options": "i"}}

# Retrieve and print actors
actors = collection.find(query)
for actor in actors:
    print(actor)

{'_id': ObjectId('652d41847230280569ccd2ab'), 'id': 1, 'name': 'Charlie Sheen'}


In [62]:
import pymongo
collection = db["movies"]

# Create a query to get all movies with only the "title" and "year" fields
projection = {"title": 1, "year": 1, "_id": 0}

# Sort the movies from recent to old by the "year" field in descending order
movies = collection.find({}, projection).sort("year", pymongo.DESCENDING)

# Retrieve and print the sorted movies
for movie in movies:
    print(movie)

{'title': 'The American President', 'year': 1995}
{'title': 'The Shawshank Redemption', 'year': 1994}
{'title': 'Wall Street', 'year': 1987}


In [63]:
# Create a query to find movies released in the 1990s
query = {"year": {"$gte": 1990, "$lt": 2000}}

# Sort the movies from old to recent by the "year" field in ascending order
movies = collection.find(query).sort("year", pymongo.ASCENDING)

# Retrieve and print the sorted movies
for movie in movies:
    print(movie)

{'_id': ObjectId('652d41847230280569ccd2b9'), 'id': 3, 'title': 'The Shawshank Redemption', 'country': 'USA', 'year': 1994}
{'_id': ObjectId('652d41847230280569ccd2b8'), 'id': 2, 'title': 'The American President', 'country': 'USA', 'year': 1995}


In [71]:
movies_collection = db["movies"]
roles_collection = db["roles"]
persons_collection = db["persons"]

# Use the $lookup operator to join the movies and roles collections
pipeline = [
    {
        "$lookup": {
            "from": "roles",           # Name of the roles collection
            "localField": "id",        # Field in the movies collection
            "foreignField": "movie_id", # Field in the roles collection
            "as": "actors"             # Alias for the joined documents
        }
    },
    {
        "$unwind": "$actors"  # Flatten the "actors" array
    },
    {
        "$lookup": {
            "from": "persons",        # Name of the persons collection
            "localField": "actors.person_id", # Field in the roles collection
            "foreignField": "id",      # Field in the persons collection
            "as": "actor_details"      # Alias for the joined persons
        }
    },
    {
        "$project": {
            "_id": 0,
            "actor": "$actor_details.name",
            "movie": "$title"
        }
    }
]

result = movies_collection.aggregate(pipeline)

for item in result:
    print(f"{item['actor']} : {item['movie']}")


['Charlie Sheen'] : Wall Street
['Michael Douglas'] : Wall Street
['Martin Sheen'] : Wall Street
['Michael Douglas'] : The American President
['Martin Sheen'] : The American President
['Morgan Freeman'] : The Shawshank Redemption


In [65]:
# Perform aggregation to get the count of movies for each actor
pipeline = [
    {
        "$lookup": {
            "from": "roles",
            "localField": "id",
            "foreignField": "person_id",
            "as": "actor_roles"
        }
    },
    {
        "$unwind": "$actor_roles"
    },
    {
        "$group": {
            "_id": "$name",
            "count": {"$sum": 1}
        }
    }
]

result = persons_collection.aggregate(pipeline)

for item in result:
    print(f"{item['_id']} acted in {item['count']} movies")

Morgan Freeman acted in 1 movies
Michael Douglas acted in 2 movies
Martin Sheen acted in 2 movies
Charlie Sheen acted in 1 movies


In [72]:
# Perform aggregation to list movies for each actor
pipeline = [
    {
        "$lookup": {
            "from": "roles",
            "localField": "id",
            "foreignField": "person_id",
            "as": "actor_roles"
        }
    },
    {
        "$unwind": "$actor_roles"
    },
    {
        "$lookup": {
            "from": "movies",
            "localField": "actor_roles.movie_id",
            "foreignField": "id",
            "as": "movie_details"
        }
    },
    {
        "$group": {
            "_id": "$name",
            "movies": {"$push": "$movie_details.title"}
        }
    }
]

result = persons_collection.aggregate(pipeline)

for item in result:
    print(f"{item['_id']} played in the following movies:")
    print([movie[0] for movie in item['movies']])

Michael Douglas played in the following movies:
['Wall Street', 'The American President']
Martin Sheen played in the following movies:
['Wall Street', 'The American President']
Charlie Sheen played in the following movies:
['Wall Street']
Morgan Freeman played in the following movies:
['The Shawshank Redemption']


In [75]:
# Find the movie "Wall Street"
wall_street_movie = movies_collection.find_one({"title": "Wall Street"})

if wall_street_movie:
    # Get the movie's ID
    movie_id = wall_street_movie["id"]

    # Use the $lookup operator to find the actors who acted in "Wall Street"
    pipeline = [
        {
            "$match": {"movie_id": movie_id}
        },
        {
            "$lookup": {
                "from": "persons",
                "localField": "person_id",
                "foreignField": "id",
                "as": "actors"
            }
        },
        {
            "$unwind": "$actors"
        },
        {
            "$project": {
                "_id": 0,
                "actor": "$actors.name"
            }
        }
    ]

    result = roles_collection.aggregate(pipeline)

    print("Persons/Actors who acted in 'Wall Street':")
    for item in result:
        print(item["actor"])
else:
    print("The movie 'Wall Street' was not found in the database.")

Persons/Actors who acted in 'Wall Street':
Charlie Sheen
Michael Douglas
Martin Sheen


In [77]:
# Find the person document for "Michael Douglas"
michael_douglas = persons_collection.find_one({"name": "Michael Douglas"})

if michael_douglas:
    # Get the actor's ID
    actor_id = michael_douglas["id"]

    # Use the $lookup and $match stages to find the movies with the actor's ID
    pipeline = [
        {
            "$lookup": {
                "from": "roles",
                "localField": "id",
                "foreignField": "movie_id",
                "as": "movie_roles"
            }
        },
        {
            "$match": {
                "movie_roles.person_id": actor_id
            }
        }
    ]

    result = movies_collection.aggregate(pipeline)

    print("Movies in which Michael Douglas has played a role:")
    for movie in result:
        print(movie["title"])
else:
    print("Michael Douglas not found in the database.")

Movies in which Michael Douglas has played a role:
Wall Street
The American President


In [78]:
# Get the count of movies in the collection
movie_count = movies_collection.count_documents({})

print("Number of movies in the database:", movie_count)

Number of movies in the database: 3


In [79]:

# Define the query to find the 'Wall Street' movie
query = {"title": "Wall Street"}

# Define the update to set the year to 2000
update = {"$set": {"year": 2000}}

# Use the update_one method to update the movie
result = movies_collection.update_one(query, update)

if result.modified_count > 0:
    print("Updated 'Wall Street' movie year to 2000")
else:
    print("No matching documents found for 'Wall Street' movie")

Updated 'Wall Street' movie year to 2000


In [80]:
# Define the filter to delete persons with names starting with 'M'
filter = {"name": {"$regex": "^M"}}

# Use the delete_many method to delete matching documents
result = persons_collection.delete_many(filter)

print(f"Deleted {result.deleted_count} persons with names starting with 'M'")

Deleted 3 persons with names starting with 'M'


In [82]:
# Add the "The Matrix" movie to the Movies collection
matrix_movie = {
    "id": 6,  # Replace with a unique ID
    "title": "The Matrix",
    "country": "USA",
    "year": 1999,
    "tagline": "Welcome to the Real World"
}
movies_collection.insert_one(matrix_movie)

# Insert the new actors to the Person collection
new_actor_list = [
    {"id": 5, "name": "Keanu Reeves", "born": 1964},
    {"id": 6, "name": "Carrie-Anne Moss", "born": 1967},
    {"id": 7, "name": "Laurence Fishburne", "born": 1960},
    {"id": 8, "name": "Hugo Weaving", "born": 1960}
]
persons_collection.insert_many(new_actor_list)

# Insert the new directors to the Person collection
new_directors = [
    {"id": 9, "name": "Lilly Wachowski", "born": 1967, "label": "Director"},
    {"id": 10, "name": "Lana Wachowski", "born": 1965, "label": "Director"}
]
persons_collection.insert_many(new_directors)

# Create the "Directed" collection and insert data into it
directed_collection = db["directed"]

directed_data = [
    {"director_id": 9, "movie_id": 6},  # Lilly Wachowski directed "The Matrix"
    {"director_id": 10, "movie_id": 6}  # Lana Wachowski directed "The Matrix"
]
directed_collection.insert_many(directed_data)


In [84]:

# Get only the directors from the Person collection
directors = persons_collection.find({"label": "Director"})

print("Directors:")
for director in directors:
    print(director["name"])


Directors:
Lilly Wachowski
Lana Wachowski


In [85]:

# Query to get persons (names and born year) who Directed "The Matrix" movie
directed_query = directed_collection.aggregate([
    {
        "$match": {"movie_id": 6}
    },
    {
        "$lookup": {
            "from": "persons",
            "localField": "director_id",
            "foreignField": "id",
            "as": "directors"
        }
    },
    {
        "$project": {
            "_id": 0,
            "name": "$directors.name",
            "born": "$directors.born"
        }
    }
])

print("\nDirectors of 'The Matrix':")
for result in directed_query:
    for director in result["name"]:
        print("Name:", director, "Born Year:", result["born"])


Directors of 'The Matrix':
Name: Lilly Wachowski Born Year: [1967]
Name: Lana Wachowski Born Year: [1965]
